In [13]:
import math as m
import numpy as np
import ipywidgets as wg
import matplotlib.pyplot as plt

# Function that Performs Antenna Pattern Calculations
def itu_omni_antenna(gain_peak, tilt, side_lobe_param, eirp):
    # Based on ITU-R Recommendation F.1336-5, published January 2019
    # Reference pattern for omnidirectional base station antenna 
    # Applicable frequency range: 400 MHz to 70 GHz
    itilt = -1*tilt
    azimuth = np.linspace(-180,180,361)
    theta_h = np.linspace(-90,90,181)
    theta_e = np.piecewise(theta_h,
                           [theta_h+itilt>=0,
                            theta_h+itilt<0],
                           [lambda theta_h: (90*(theta_h+itilt))/(90+itilt),
                            lambda theta_h: (90*(theta_h+itilt))/(90-itilt)]
                          )
    threedB_BW = 107.6*10**(-0.1*gain_peak)
    sidelobe_bump = threedB_BW*np.sqrt(1-((1/1.2)*np.log10(side_lobe_param+1)))
    gain_pattern = np.piecewise(theta_e,
                        [np.abs(theta_e)<sidelobe_bump,
                         (np.abs(theta_e)>=sidelobe_bump) & (np.abs(theta_e)<threedB_BW),
                         np.abs(theta_e)>=threedB_BW],
                        [lambda theta_e: gain_peak - 12*(theta_e/threedB_BW)**2,
                         lambda theta_e: gain_peak - 12 + 10*np.log10(side_lobe_param+1),
                         lambda theta_e: gain_peak - 12 + 10*np.log10((abs(theta_e)/threedB_BW)**-1.5 + side_lobe_param)]
                       )
    heat_pattern = np.tile(gain_pattern, (len(azimuth),1)).T
    heat_values = (eirp - gain_peak) + heat_pattern
    return gain_pattern, heat_values, theta_h

# Function that Updates the Patterns in the Output Display
def update_patterns(gain_peak, tilt, side_lobe_param, eirp):
    gain_nominal, heat_nominal, elevation = itu_omni_antenna(gain_peak, 0, 0, eirp)
    gain, heat, elevation = itu_omni_antenna(gain_peak, tilt, side_lobe_param, eirp)
    
    fig = plt.figure(figsize=(8, 16))
    
    # Cartesian Plot
    ax1 = fig.add_subplot(311)
    ax1.plot(gain_nominal, elevation, color='black', linewidth=1, linestyle=':', label='nominal gain')
    ax1.plot(gain, elevation, color='blue', linewidth=1, linestyle='-', label='with sidelobes and tilt')
    ax1.set_yticks(np.linspace(min(elevation), max(elevation), 13))
    ax1.set_ylabel('Elevation Angle (deg.)')
    ax1.set_xlabel('Gain (dBi)')
    ax1.set_title('ITU-R F.1336.5 Omnidirectional Antenna Gain')
    ax1.legend()
    ax1.grid(True)
    
    # Polar Plot
    theta_rad = np.radians(elevation)
    ax2 = fig.add_subplot(312, projection='polar')
    ax2.plot(theta_rad, gain_nominal, color='black', linewidth=1, linestyle=':', label='nominal gain')
    ax2.plot(theta_rad, gain, color='blue', linewidth=1, linestyle='-', label='with sidelobes and tilt')
    ax2.set_thetamin(-90)
    ax2.set_thetamax(90)
    ax2.set_thetagrids(np.arange(-90, 91, 30))
    ax2.set_rlim(-30, gain_peak)
    ax2.set_ylabel("Gain (dBi)", labelpad=-30)
    ax2.set_title("ITU-R F.1336.5 Omnidirectional Antenna Gain")
    ax2.legend(loc='upper right', fontsize=8)
    ax2.annotate(f'Peak: {gain_peak:.1f} dBi',
                 fontsize='9', color='black',
                 xy=(0, gain_peak), xytext=(0.88, 0.6),
                 textcoords='axes fraction', ha='center', va='top')
    
    # Heatmap
    ax3 = fig.add_subplot(313)
    im = ax3.imshow(heat, extent=[-180,180,-90,90],
                    aspect='auto', origin='lower',
                    cmap='jet', vmin=m.floor(np.min(heat_nominal)), vmax=eirp
                    )
    ax3.set_xlabel('Azimuth Angle (deg.)')
    ax3.set_ylabel('Elevation Angle (deg.)')
    ax3.set_title('Spatial Emissions Pattern w/ ITU-R F.1336.5 Omnidirectional Antenna')
    plt.colorbar(im, ax=ax3, label='EIRP (dBm/10MHz)')
 
    plt.tight_layout()
    plot_output = wg.Output()
    with plot_output:
        display(fig)
    plt.close(fig)
    return plot_output

# Update Function: Triggers the patterns to update when any of the inputs are changed
def update(*args):
    if power_select.value == 'EIRP':
        eirp = power_widget.value    
    elif power_select.value == 'Conducted Power':
        eirp = power_widget.value + gain_widget.value
    plot_output = update_patterns(gain_widget.value,
                                  tilt_widget.value,
                                  sidelobe_widget.value,
                                  eirp
                                 )
    patterns.clear_output(wait=True)
    with patterns:
        display(plot_output)

# Create Input Widgets
styl = {'description_width':'initial'}
box1 = wg.Layout(width='210px')
box2 = wg.Layout(width='150px')
heading_ant = wg.HTML(value="<b>Antenna Attributes</b>")
heading_tx = wg.HTML(value="<b>Transmitter Attributes</b>")
gain_widget = wg.BoundedFloatText(description='Peak Antenna Gain (dBi):',value=8.5,min=0,max=15,step=0.5,style=styl,layout=box1)
tilt_widget = wg.BoundedFloatText(description='Vertical Tilt (degrees):',value=0,min=-10,max=10,step=0.1,style=styl,layout=box1)
sidelobe_widget = wg.BoundedFloatText(description='Sidelobe Parameter (k):',value=0.7,min=0,max=5,step=0.1,style=styl,layout=box1)
power_select = wg.RadioButtons(options=['EIRP','Conducted Power'],value='EIRP')
power_widget = wg.FloatText(description='(dBm/10MHz)',value=47,layout=box2)

# Monitor Widgets for Changes to User Input
gain_widget.observe(update,names='value')
tilt_widget.observe(update,names='value')
sidelobe_widget.observe(update,names='value')
power_select.observe(update,names='value')
power_widget.observe(update,names='value')

# Display the Widgets
antenna_parameters = wg.VBox([heading_ant, gain_widget, tilt_widget, sidelobe_widget])
transmitter_parameters = wg.VBox([heading_tx, power_select, power_widget])
grid = wg.GridspecLayout(1,2,width='650px')
grid[0,0] = antenna_parameters
grid[0,1] = transmitter_parameters
display(grid)

# Display the Patterns
patterns = wg.Output()
display(patterns)

update()

GridspecLayout(children=(VBox(children=(HTML(value='<b>Antenna Attributes</b>'), BoundedFloatText(value=8.5, d…

Output()